In [21]:
import numpy as np
X = np.random.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import MultinomialNB, GaussianNB
clf =  MultinomialNB()
clf.fit(X, y)

test = X[0]
test[1] = 0
test = np.random.randint(5, size=(100))

print(clf.predict_proba(test))

[[  7.94771063e-06   9.90030296e-01   9.86937379e-03   5.95542608e-08
    7.37611133e-05   1.85614682e-05]]


In [47]:
np.ones((10,1))

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [125]:
### normcdf
import numpy as np
from scipy.special import erfc
import math

def normcdfM(x, m=None, s=None):
    if m==None and s==None:
        z = x
    if s==None:
        z = x-m
    else:
        z = (x-m)/s
        
    return 0.5*erfc(-z/math.sqrt(2))
    

In [126]:
### normpdf

import numpy as np
import math

def normcdfM(x, m=0, s=1):
    z = (x-m)/s
    
    o = 1/math.sqrt(2*math.pi()*(s**2))
    return o*math.exp(-((x-m)**2)/(2*(s**2)))
    

In [145]:
##tmean

import numpy as np

def tmean(m, index_max, n_samps):
    K = m.shape[0]
    u = np.random.randn(n_samps,1)

    t = m[index_max]*np.ones((K,1)) - m ## error?
    tr = t
    t = np.delete(t, index_max, 0)
    # t[index_max] = [] TEST
    
    print m.shape
    print u.shape
    print K
    print t.shape # wrong
    print n_samps
    
    s = np.tile(u, (1, K-1)) + np.tile(t,(1, n_samps)).conj().T
    z = np.mean(np.prod(safenormcdf(s.conj().T), 0))

    for r in range(0, K):
        sr = tile(u, (1, K)) + tile(tr, (1, n_samps)).conj().T
        print sr.take([r, index_max], axis=1) # TEST
        nr = np.mean(safenormpdf(u.conj().T + m[index_max] 
                                 - m[r])*np.prod(safenormcdf(sr.conj().T), 1))
        
        if r == index_max:
            tm[r] = 0
        else:
            tm[r] = m[r] - nr/z
            
    tm[index_max] = np.sum(m, axis=0) - np.sum(tm, axis=0)
    tm = tm.conj().T                    
            
    return tm, z
                                                               

def safenormcdf(x):
    thresh=-10;
    x[np.nonzero(x<thresh)] = thresh
    return normcdfM(x)

def safenormpdf(x):
    thresh=35;
    x[np.nonzero(x<-thresh)] = -thresh
    x[np.nonzero(x>thresh)] = thresh
    return normpdfM(x)
        

In [146]:
import numpy as np

def dist(X,Y,Z): # X, Y matrices, Z vector
    nx = X.shape[0]
    ny = Y.shape[0]

    distance= (np.sum((X**2)*Z, 1)*np.ones((1,ny))+
               np.ones((nx,1))*(np.sum((X**2)*Z, 1).conj().T) - 2*(np.dot(X*Z, Y.conj().T)))
        
    return distance


def VarMultProbRegGP(X, t, X_test=0, t_test=0, theta=0, theta_estimate=0, 
                     nos_its=0, kernel_Type=0, poly_kernel_power=0, thresh=0):

    SMALL_NOS = 1e-10

    nos_samps_tg = 1000

    nos_samps_is = 1000

    sigma = 1e-6

    tau = 1e-6

    C = np.max(t)
    N, D = X.shape
    Y = np.random.randn(N, C)
    M = np.random.randn(N, C)
    beta = theta
    Theta = np.diag(theta)

    psi = np.ones((1, theta.size)) 
    In = np.eye(N)
    Ic = np.eye(C)

    diff = 1e100
    its = 0

    K = np.exp(-dist(X,X,Theta)) + np.eye(N)*SMALL_NOS; # TODO: implement create kernel function

    iK = np.linalg.inv(K+In)
    Ki = K*iK
    
    THETA = []
    #LOWER_BOUND = [-1e-3]
    PL = []
    
    test_err = []
    while its<nos_its and diff>thresh:
        its += 1
        
        for k in range(0, C):
            M[:,k] = np.dot(Ki, Y[:,k])
            
        for n in range(0, N):
            a, z = tmean(M[n,].conj().T, t[n], nos_samps_tg)
            Y[n,] = a.conj().T
    
    
    
X = np.random.randint(5, size=(6, 100))
theta = np.random.rand(1, 100)
y = np.array([1, 2, 3, 4, 5, 6])
VarMultProbRegGP(X, y, theta=theta, nos_its=100, thresh = 0.01)    



(6,)
(1000, 1)
6
(5, 6)
1000


ValueError: operands could not be broadcast together with shapes (1000,5) (6000,5) 